# Link al libro que estoy usando para armar el LLM: [Build a Large Language Model from Scratch - Sebastian Raschka](https://drive.google.com/file/d/1WkucQZgK4RENhGG_lm75yRqcQWw-Pxzk/view?usp=sharing)


In [24]:
with open('./samples/llm_from_scratch/the-verdict.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()

print(f'Número total de caracteres: {len(raw_text)}.')
print(raw_text[:99])

Número total de caracteres: 20479.
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Hace esto, línea por línea:

1.

```python
with open('./sample_data/llm_from_scratch/the-verdict.txt', 'r', encoding='utf-8') as file:
```

* `open(path, mode, encoding=...)`: abre un archivo y devuelve un objeto fichero.

  * `path`: `'./sample_data/llm_from_scratch/the-verdict.txt'`. Ruta relativa al **directorio de trabajo actual**. En Colab suele ser `/content`. `./` apunta ahí.
  * `mode='r'`: modo lectura de texto. No crea ni modifica el archivo.
  * `encoding='utf-8'`: decodifica bytes a texto Unicode con UTF-8. Evita errores de acentos.
* `with ... as file:` usa un **context manager**. Garantiza cierre del archivo al salir del bloque, incluso si hay excepciones. Evita fugas de descriptores.

2.

```python
    raw_text = file.read()
```

* `file.read()` lee **todo** el contenido en memoria y retorna un `str` (Unicode). Complejidad O(n).
* Útil para archivos pequeños/medianos. Para archivos muy grandes, preferir lectura por trozos o iterar líneas.

3.

```python
print(f'Número total de caracteres: {len(raw_text)}.')
```

* `len(raw_text)`: número de **caracteres Unicode** del `str`. No son bytes.
* Un “carácter” aquí es un **code point**. Algunos grafemas visibles pueden componerse de varios code points; `len` contaría más de uno en ese caso.
* `f'...'`: f-string. Evalúa expresiones dentro de `{}` y las inserta en el texto.

4.

```python
print(raw_text[:99])
```

* `raw_text[:99]`: **slice** desde el inicio hasta el índice 99 **excluido**. Devuelve como mucho 99 caracteres. Si el texto es más corto, no falla.
* Útil para previsualizar el inicio del archivo.

Notas prácticas en Colab:

* Verifica que el archivo exista en esa ruta. Si falla, `FileNotFoundError`.
* Si ves errores de codificación, confirma UTF-8 del archivo. Alternativa: `encoding='utf-8-sig'` si trae BOM, o `errors='replace'` para caracteres inválidos.
* Ruta alternativa robusta:

  ```python
  from pathlib import Path
  p = Path('sample_data/llm_from_scratch/the-verdict.txt')
  with p.open('r', encoding='utf-8') as f:
      raw_text = f.read()
  ```
* Si necesitas memoria estable, no uses `read()` con archivos muy grandes. Mejor:

  ```python
  size = 0
  with open(path, 'r', encoding='utf-8') as f:
      for line in f:
          size += len(line)
  ```

In [25]:
import re

text = 'Hey, you. This, is a test.'
result = re.split(r'\s', text)
print(result)

['Hey,', 'you.', 'This,', 'is', 'a', 'test.']


Explicación precisa línea por línea:

1.

```python
import re
```

* Importa el módulo estándar **`re`** (regular expressions). Permite búsqueda, coincidencia y manipulación de texto mediante **expresiones regulares**.
* Internamente compila patrones a autómatas finitos optimizados.

2.

```python
text = 'Hey, you. This, is a test.'
```

* Declara una variable `text` tipo `str` con el contenido literal indicado.
* Contiene palabras, comas, puntos y espacios.

3.

```python
result = re.split(r'\s', text)
```

* `re.split(pattern, string)` divide `string` en una lista, usando **las coincidencias del patrón** como separadores.
* `r'\s'` es un *raw string literal*: el prefijo `r` indica que `\` se trata como carácter literal y no como escape de Python.
* En una expresión regular, `\s` coincide con **cualquier carácter de espacio en blanco**: espacio, tabulador, salto de línea, retorno de carro o tab vertical.
* Por tanto, esta instrucción separa el texto cada vez que encuentra un espacio o cualquier carácter blanco.
* Devuelve una lista de subcadenas sin los delimitadores (los espacios no se incluyen en la salida).

4.

```python
print(result)
```

* Imprime la lista resultante.
* Salida esperada:

  ```
  ['Hey,', 'you.', 'This,', 'is', 'a', 'test.']
  ```

  Cada palabra o palabra con puntuación permanece unida porque `re.split` solo corta por espacios.

Detalles adicionales:

* Si el texto tuviera varios espacios seguidos, el resultado incluiría elementos vacíos (`''`) donde se encuentran separadores consecutivos.
* Ejemplo: `'a  b'` → `['a', '', 'b']`.
* Para evitar vacíos: `re.split(r'\s+', text.strip())`, que divide por uno o más espacios y elimina espacios iniciales/finales.


In [26]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hey', ',', '', ' ', 'you', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


Explicación detallada:

1.

```python
result = re.split(r'([,.]|\s)', text)
```

* `re.split(patrón, texto)` divide `texto` usando las **coincidencias del patrón** como separadores.

* El patrón `r'([,.]|\s)'` se interpreta así:

  * `r'...'`: *raw string literal* evita que `\s` sea procesado como escape por Python.
  * `(...)`: **grupo de captura**. Si el patrón contiene paréntesis, los separadores encontrados se **conservan** en la lista de salida.
  * `[,.]`: clase de caracteres; coincide con **una coma (`,`) o un punto (`.`)**.
  * `|`: operador **OR** lógico en expresiones regulares.
  * `\s`: coincide con **cualquier espacio en blanco** (espacio, tabulador, salto de línea, etc.).

* En conjunto, el patrón significa:
  “Divide el texto cada vez que aparezca una coma, un punto o un espacio, e incluye ese carácter separador como elemento independiente en el resultado.”

* El grupo de captura `(...)` es lo que diferencia este ejemplo del anterior: sin él, los separadores se descartarían; con él, se **mantienen** en la lista.

2.

```python
print(result)
```

* Muestra la lista completa generada.
* Para `text = 'Hey, you. This, is a test.'`, el resultado es:

  ```
  ['Hey', ',', '', ' ', 'you', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
  ```

Análisis del resultado:

* Cada palabra o signo está separado individualmente.
* Los elementos vacíos (`''`) aparecen porque `re.split` puede generar subcadenas vacías cuando dos separadores se suceden (por ejemplo, una coma seguida de un espacio).
* Los separadores detectados (`,`, `.`, `' '`) se mantienen en la lista.

Uso posterior:

* Estos resultados suelen limpiarse con una lista por comprensión:

  ```python
  result = [item for item in result if item.strip()]
  ```

  que elimina los vacíos y deja solo palabras y signos de puntuación separados.


In [27]:
result = [item for item in result if item.strip()]
print(result)

['Hey', ',', 'you', '.', 'This', ',', 'is', 'a', 'test', '.']


Explicación exacta:

1.

```python
result = [item for item in result if item.strip()]
```

* Es una **lista por comprensión** (*list comprehension*).
* Recorre cada elemento `item` dentro de la lista `result` anterior.
* `item.strip()` ejecuta el método `str.strip()` de Python, que:

  * Elimina todos los caracteres de espacio en blanco al inicio y final del string (`' '`, `\t`, `\n`, etc.).
  * Devuelve el texto sin esos espacios.
* En un contexto booleano, un string vacío `''` evalúa como `False`.
* Así, la condición `if item.strip()` **filtra** y mantiene solo los elementos cuyo resultado no está vacío.
* Resultado: se eliminan todos los tokens vacíos (`''`) generados por separadores consecutivos o espacios redundantes.

2.

```python
print(result)
```

* Imprime la nueva lista limpia, sin elementos vacíos.
* Con `text = 'Hey, you. This, is a test.'`, la salida será:

  ```
  ['Hey', ',', 'you', '.', 'This', ',', 'is', 'a', 'test', '.']
  ```

Resultado final:

* Cada palabra y signo de puntuación es un token independiente.
* Se conserva el orden original del texto.
* Ya no hay espacios ni cadenas vacías.

Importancia:

* Este filtrado prepara el texto para construir vocabularios o convertir tokens en IDs numéricos, pasos previos al entrenamiento de un modelo de lenguaje.


In [29]:
text = 'Hey, you. Is ths-- a test?'
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

result = [item for item in result if item.strip()]
print(result)

['Hey', ',', 'you', '.', 'Is', 'ths', '--', 'a', 'test', '?']


Explicación exhaustiva:

1.

```python
text = 'Hey, you. Is ths-- a test?'
```

* Define la variable `text` como una cadena (`str`).
* Contiene palabras, signos de puntuación, un doble guion (`--`) y espacios.
* Este tipo de texto se usa para probar una tokenización más compleja.

---

2.

```python
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
```

* Usa el método `re.split()` del módulo `re` para **dividir el texto** con una expresión regular avanzada.

* Desglose del patrón `r'([,.:;?_!"()\']|--|\s)'`:

  * `r'...'`: *raw string literal*, evita que Python interprete `\` como secuencia de escape.
  * `(...)`: **grupo de captura**, conserva los separadores en la salida.
  * `[ ,.:;?_!"()\' ]`: **clase de caracteres**, coincide con cualquiera de estos signos de puntuación:
    `, . : ; ? _ ! " ( ) '`
  * `|--`: el operador `|` indica “o”. Aquí captura explícitamente la secuencia de **doble guion** `--`.
  * `|\s`: también divide donde haya **espacios en blanco** (espacios, tabuladores o saltos de línea).

* En conjunto, el patrón significa:
  “Divide el texto en cada espacio o signo de puntuación del listado, y conserva esos signos como elementos separados en la lista resultante.”

---

3.

```python
result = [item for item in result if item.strip()]
```

* Lista por comprensión para eliminar tokens vacíos (`''`) o compuestos solo por espacios.
* `item.strip()` elimina espacios al inicio y al final; si el resultado está vacío, se descarta.
* Resultado: una lista de tokens limpios (palabras y signos).

---

4.

```python
print(result)
```

* Imprime la lista final de tokens.
* Salida esperada:

  ```
  ['Hey', ',', 'you', '.', 'Is', 'ths', '--', 'a', 'test', '?']
  ```

---

**Resumen funcional:**

* La expresión regular separa palabras y signos de puntuación en elementos individuales.
* El filtrado posterior elimina espacios vacíos.
* El resultado es una **tokenización básica** útil para construir vocabularios o convertir texto en secuencias de IDs numéricos en el preprocesamiento de un LLM.


# Tokenizando el texto



In [30]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


Explicación exhaustiva, línea por línea:

---

### 1)

```python
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
```

**Propósito:**
Tokenizar el texto completo cargado en `raw_text` (por ejemplo, el cuento *The Verdict*).
Divide el texto en unidades mínimas (*tokens*) —palabras y signos de puntuación— que luego se usarán como entrada para un modelo de lenguaje.

**Desglose técnico:**

* **`re.split(patrón, cadena)`**
  Función del módulo estándar `re` (*regular expressions*) que divide una cadena (`cadena`) en una lista, usando las coincidencias del `patrón` como delimitadores.
  A diferencia de `str.split()`, que solo separa por espacios, `re.split()` puede usar expresiones regulares complejas y permite conservar los delimitadores si están dentro de **paréntesis de captura**.

* **`r'([,.:;?_!"()\']|--|\s)'`**
  Es un *raw string literal* (`r'...'`), lo que evita que Python interprete `\` como secuencia de escape (por ejemplo, `\s` no se convierte en “espacio”, sino que se pasa literalmente a la expresión regular).

  Desglose del patrón:

  * **`(...)`** → Grupo de captura. Todo lo que coincida dentro de este grupo se incluye en la lista de salida.
  * **`[,.:;?_!"()\' ]`** → Clase de caracteres que captura **cualquiera** de los signos de puntuación listados:
    `, . : ; ? _ ! " ( ) '`
  * **`|--`** → Alternativa literal que detecta la secuencia exacta de **doble guion** `--` (frecuente en narrativa).
  * **`|\s`** → Detecta cualquier carácter de espacio en blanco (`' '`, `\t`, `\n`, etc.).

  En conjunto, el patrón significa:

  > Divide el texto cada vez que aparezca una coma, punto, signo de interrogación, doble guion o espacio, e incluye esos separadores en la lista resultante.

**Resultado intermedio:**
La variable `preprocessed` se convierte en una lista en la que cada elemento puede ser:

* una palabra,
* un signo de puntuación aislado (`,`, `.`, `--`, etc.), o
* una cadena vacía si se encuentran separadores consecutivos.

---

### 2)

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

**Propósito:**
Limpiar la lista de tokens eliminando elementos vacíos o compuestos solo por espacios.

**Explicación técnica:**

* `item.strip()` ejecuta el método `str.strip()` sobre cada `item`:

  * Quita caracteres de espacio en blanco al inicio y final del string.
  * Devuelve una nueva cadena sin esos caracteres.
* En un contexto booleano, un string vacío (`''`) se evalúa como `False`.
* Por tanto, la condición `if item.strip()` **filtra** y conserva solo los elementos con contenido textual o de puntuación válido.
* El resultado final es una lista limpia con todos los tokens “reales” que el modelo procesará.

**Complejidad temporal:**
O(n) respecto al número de tokens, ya que el filtrado recorre cada elemento una vez.

---

### 3)

```python
print(len(preprocessed))
```

**Propósito:**
Mostrar cuántos tokens (palabras y signos) contiene el texto tras la tokenización y limpieza.

**Detalles técnicos:**

* `len(preprocessed)` devuelve el número de elementos de la lista (un entero).
* Este número equivale a la **longitud del corpus tokenizado**.
* Permite comprobar si la división del texto produjo una cantidad razonable de tokens.

  * Por ejemplo, el cuento *The Verdict* produce ~4 690 tokens sin espacios.

**Consideraciones de uso:**

* Este conteo no equivale a palabras “semánticas”, ya que incluye signos de puntuación como tokens separados.
* El resultado sirve para determinar el tamaño del vocabulario base que se usará en la siguiente etapa (construcción del diccionario de tokens únicos).

---

**Resumen funcional global:**

1. Divide el texto en palabras y signos mediante una expresión regular amplia.
2. Elimina vacíos y espacios innecesarios.
3. Devuelve una lista limpia de tokens que representa la forma mínima procesable del corpus textual.
4. Muestra el número total de tokens para verificar la segmentación antes de construir el vocabulario del modelo.


# 2.3 Convirtiendo tokens en ID's

In [31]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token: integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
  print(item)
  if i > 50:
    break


1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


Explicación exhaustiva, línea por línea:

---

### 1)

```python
all_words = sorted(set(preprocessed))
```

**Objetivo:**
Extraer el **vocabulario único** del corpus tokenizado `preprocessed` y ordenarlo alfabéticamente.

**Desglose técnico:**

* `set(preprocessed)`

  * Convierte la lista `preprocessed` (que contiene tokens repetidos) en un **conjunto** (`set`), una estructura de datos desordenada y sin duplicados.
  * Cada elemento del conjunto es **único**, por lo que esta operación elimina repeticiones.
  * Implementación interna: los `set` de Python están basados en **tablas hash**.

    * La inserción y la verificación de unicidad (`hashing`) tienen complejidad media **O(1)**.
  * Complejidad total de conversión: **O(n)**, donde *n* es el número de tokens en `preprocessed`.

* `sorted(...)`

  * Toma el conjunto resultante y devuelve una **lista ordenada lexicográficamente**.
  * La ordenación se realiza con el algoritmo **Timsort** (fusión adaptativa, estable y de complejidad O(n log n)).
  * El orden lexicográfico depende de los códigos Unicode de los caracteres, por lo que letras mayúsculas (`'A'`) aparecen antes que minúsculas (`'a'`).

**Resultado:**
Una lista (`list`) llamada `all_words` que contiene todos los **tokens únicos** (palabras y signos de puntuación), ordenados.

---

### 2)

```python
vocab_size = len(all_words)
```

**Objetivo:**
Calcular el tamaño del vocabulario, es decir, cuántos tokens distintos existen.

**Detalles:**

* `len()` accede al atributo de longitud de la lista (`PyObject_VAR_HEAD->ob_size` en CPython).
* Complejidad temporal: **O(1)**, ya que `len()` no recorre la lista; accede directamente al valor almacenado internamente.
* `vocab_size` es un entero (`int`) que representa el número total de tokens únicos.

---

### 3)

```python
print(vocab_size)
```

**Propósito:**
Mostrar la cantidad de elementos únicos (palabras, signos y símbolos) en el vocabulario.

* En el ejemplo del libro, para *The Verdict*, este valor es aproximadamente **1 130** tokens.
* Este número será fundamental para:

  * definir el tamaño de las matrices de embeddings,
  * establecer la dimensionalidad de las capas de entrada y salida del modelo (por ejemplo, `nn.Embedding(vocab_size, embed_dim)` en PyTorch).

---

### 4)

```python
vocab = {token: integer for integer, token in enumerate(all_words)}
```

**Objetivo:**
Crear un **diccionario de mapeo** entre tokens y sus identificadores numéricos (*token IDs*).

**Desglose técnico:**

* **`enumerate(all_words)`**

  * Genera un iterador que produce tuplas `(índice, elemento)` para cada token en `all_words`.
  * Por defecto, la enumeración comienza en 0.
  * Ejemplo: `[('!', 0), ('"', 1), ("'", 2), ('(', 3), ...]`.
  * Complejidad: **O(n)**, donde *n* es el número de tokens únicos.

* **Comprensión de diccionario `{token: integer for integer, token in enumerate(all_words)}`**

  * Recorre cada par `(integer, token)` producido por `enumerate`.
  * Asigna el token (`token`) como **clave** y el número (`integer`) como **valor**.
  * Resultado: un diccionario que implementa la función ( f : \text{token} \mapsto \text{ID entero} ).

**Propiedades del diccionario (`dict`):**

* En Python 3.7+, los diccionarios **mantienen el orden de inserción**, por lo que el orden coincide con la lista ordenada `all_words`.
* Internamente usan **tablas hash** (resolución abierta), lo que permite:

  * Búsqueda promedio: **O(1)**.
  * Inserción promedio: **O(1)**.
* Tamaño de `vocab` = `vocab_size`.

**Importancia:**
Este mapeo es el núcleo del proceso de **tokenización numérica**, que permite convertir texto en tensores de enteros antes del entrenamiento del modelo.

---

### 5)

```python
for i, item in enumerate(vocab.items()):
  print(item)
  if i > 50:
    break
```

**Objetivo:**
Verificar visualmente las primeras asignaciones del diccionario `vocab`.

**Desglose técnico:**

* **`vocab.items()`**

  * Devuelve un objeto *view* que itera sobre las **tuplas (clave, valor)** del diccionario.
  * Ejemplo: `('!', 0), ('"', 1), ("'", 2), ('(', 3), ...`.
  * Complejidad: **O(1)** por acceso, **O(n)** por recorrido completo.

* **`enumerate(...)`**

  * Agrega un contador `i` para poder detener el bucle después de imprimir 51 pares.

* **`print(item)`**

  * Muestra en consola cada par `(token, id)`.

* **Condición `if i > 50: break`**

  * Interrumpe el bucle tras 51 iteraciones (índices de 0 a 50).
  * Esto evita imprimir el vocabulario completo, que puede tener cientos o miles de tokens.

---

### **Resumen funcional completo:**

1. Se eliminan duplicados del corpus tokenizado.
2. Se ordenan los tokens para generar una lista estable y reproducible (`all_words`).
3. Se calcula el tamaño del vocabulario (`vocab_size`).
4. Se crea un diccionario `vocab` que asigna a cada token un identificador entero único, base de la codificación del texto.
5. Se imprime una muestra del mapeo para validación visual.

---

**Resultado conceptual:**
Este bloque construye la **primera capa de representación simbólica** de un LLM:

> un espacio discreto de vocabulario donde cada símbolo del lenguaje natural queda asociado a un índice entero, preparando el terreno para el embedding vectorial que convertirá estos índices en representaciones continuas en la siguiente etapa del pipeline.


In [32]:
class SimpleTokenizerV1:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()} # ('his', 51) i= 51, s= his

  def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = ' '.join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

    return text


Explicación exhaustiva del código y de los principios que implementa:

---

## **Clase `SimpleTokenizerV1`**

Esta clase define un **tokenizador mínimo y bidireccional** que convierte texto en secuencias numéricas (*encoding*) y viceversa (*decoding*).
Opera sobre un vocabulario (`vocab`) previamente construido que asigna **tokens → IDs numéricos**.

La implementación ilustra el flujo esencial de un **preprocesador de texto para LLMs** antes de usar técnicas más avanzadas como *byte pair encoding (BPE)*.

---

### **1) Definición de clase**

```python
class SimpleTokenizerV1:
```

* Define un **objeto de tipo clase** en Python que agrupa datos (atributos) y comportamiento (métodos).
* Sirve como plantilla para crear instancias específicas del tokenizador.
* En tiempo de ejecución, cada instancia mantiene su propio estado interno (diccionarios de mapeo, etc.).

---

### **2) Método `__init__`**

```python
def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()}  # ('his', 51) → i=51, s='his'
```

**Propósito:**
Inicializa la instancia y crea los dos diccionarios de mapeo complementarios:

1. **`str_to_int`**: de token (texto) a ID (entero).
2. **`int_to_str`**: de ID (entero) a token (texto).

**Detalles técnicos:**

* `__init__` es el **constructor** especial de Python. Se ejecuta automáticamente al crear un nuevo objeto:

  ```python
  tokenizer = SimpleTokenizerV1(vocab)
  ```

  Este llama implícitamente a `SimpleTokenizerV1.__init__(tokenizer, vocab)`.

* `self.str_to_int = vocab`

  * Almacena el diccionario que mapea cada token único del vocabulario a un entero (ID).
  * Ejemplo: `{'!', 0, '"': 1, "'", 2, ..., 'his': 51}`.

* `self.int_to_str = {i: s for s, i in vocab.items()}`

  * **Inversión del mapeo.**
  * `vocab.items()` produce pares `(token, id)`.
  * La comprensión de diccionario los invierte para formar `(id, token)`.
  * Ejemplo: `{51: 'his', 52: 'her', ...}`.
  * Esto permite reconstruir texto desde secuencias de IDs en la operación *decode*.
  * Complejidad de construcción: **O(n)** en el número de tokens.

---

### **3) Método `encode`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

**Función general:**
Convierte texto plano en una secuencia de **identificadores numéricos** según el vocabulario.

**Etapas detalladas:**

1. **Tokenización inicial**

   ```python
   preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
   ```

   * Usa una expresión regular para dividir el texto en **palabras y signos de puntuación**.
   * Patrón:

     * `(...)`: grupo de captura (mantiene los delimitadores).
     * `[,.?_!"()']`: signos a separar.
     * `|--`: doble guion literal.
     * `|\s`: espacio o tabulador.
   * Ejemplo:
     `'Hey, you!' → ['Hey', ',', '', ' ', 'you', '!', '']`

2. **Limpieza de tokens**

   ```python
   preprocessed = [item.strip() for item in preprocessed if item.strip()]
   ```

   * `str.strip()` elimina espacios en blanco al inicio y fin de cada token.
   * La condición `if item.strip()` elimina elementos vacíos (`''`).
   * Resultado limpio: `['Hey', ',', 'you', '!']`.

3. **Conversión a IDs**

   ```python
   ids = [self.str_to_int[s] for s in preprocessed]
   ```

   * Sustituye cada token textual por su ID numérico.
   * Acceso al diccionario: O(1) promedio por búsqueda (tabla hash).
   * Si un token no está en `vocab`, esta versión **lanza KeyError** (no maneja desconocidos).
   * Ejemplo:
     Si `vocab['Hey'] = 45`, `vocab[','] = 3`, la salida será `[45, 3, 120, 5]`.

4. **Retorno**

   * Devuelve una lista de enteros (`List[int]`).
   * Este formato es compatible con frameworks como PyTorch (`torch.tensor(ids)`).

---

### **4) Método `decode`**

```python
def decode(self, ids):
    text = ' '.join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

**Función general:**
Transforma una secuencia numérica de tokens de vuelta a texto legible.

**Etapas detalladas:**

1. **Conversión de IDs a texto**

   ```python
   [self.int_to_str[i] for i in ids]
   ```

   * Usa el diccionario inverso `int_to_str` para reconstruir los tokens.
   * Genera una lista de strings.
   * Si un ID no está en el diccionario, se lanzará un `KeyError`.

2. **Concatenación con espacios**

   ```python
   ' '.join([...])
   ```

   * Une todos los tokens con un **espacio** entre ellos.
   * Ejemplo: `['Hey', ',', 'you', '!'] → 'Hey , you !'`.

3. **Corrección de espaciado antes de signos**

   ```python
   re.sub(r'\s+([,.?!"()\'])', r'\1', text)
   ```

   * Busca espacios seguidos de signos de puntuación.
   * Patrón:

     * `\s+`: uno o más espacios.
     * `([,.?!"()'])`: grupo de captura con signos.
   * Reemplazo `r'\1'`: mantiene solo el signo, eliminando el espacio anterior.
   * Resultado: `'Hey, you!'` (sin espacio extra antes de `,` o `!`).
   * Complejidad: O(n) sobre la longitud del texto.

4. **Retorno**

   * Devuelve el texto corregido y reconstruido como `str`.

---

### **5) Ejemplo de uso**

```python
tokenizer = SimpleTokenizerV1(vocab)
text = "It's the last he painted, you know."
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))
```

**Flujo interno:**

1. `encode()` → tokeniza → mapea a IDs.
2. `decode()` → convierte IDs → reconstruye texto.

**Resultado esperado:**

```python
[12, 45, 78, 91, 8, 120, 5, 32, 14]  # ejemplo de IDs
"It's the last he painted, you know."
```

---

### **6) Limitaciones del diseño V1**

* No maneja **palabras desconocidas**: si un token no está en el vocabulario, lanza `KeyError`.
* No incluye **tokens especiales** (`<|unk|>`, `<|endoftext|>`, etc.).
* No conserva espacios exactos, por lo que pierde formato o saltos de línea.
* Sirve como **prototipo conceptual** antes de evolucionar hacia `SimpleTokenizerV2` y finalmente hacia el **BPE tokenizer** usado en GPT.

---

### **Resumen conceptual**

| Etapa    | Entrada      | Proceso                             | Salida        | Complejidad |
| -------- | ------------ | ----------------------------------- | ------------- | ----------- |
| `encode` | texto crudo  | regex + limpieza + mapeo            | lista de IDs  | O(n)        |
| `decode` | lista de IDs | reconstrucción + ajuste de espacios | texto legible | O(n)        |

El objetivo pedagógico de `SimpleTokenizerV1` es mostrar **cómo un LLM traduce lenguaje natural a un espacio discreto numérico** antes de aplicar embeddings y atención. Es la base de la representación simbólica que alimenta las capas iniciales del modelo transformer.


In [36]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know, "Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


Explicación exhaustiva, línea por línea:

---

### **1)**

```python
tokenizer = SimpleTokenizerV1(vocab)
```

**Propósito:**
Crea una **instancia** de la clase `SimpleTokenizerV1` definida previamente, usando como entrada el diccionario `vocab`.

**Detalles técnicos:**

* `SimpleTokenizerV1(vocab)` ejecuta internamente:

  ```python
  SimpleTokenizerV1.__init__(tokenizer, vocab)
  ```

  donde `tokenizer` es el nuevo objeto creado en memoria (tipo `SimpleTokenizerV1`).

* En el constructor (`__init__`):

  * `self.str_to_int = vocab` asigna el vocabulario base (tokens → IDs).
  * `self.int_to_str = {i: s for s, i in vocab.items()}` crea el diccionario inverso (IDs → tokens).

**Estructura interna resultante:**

* `tokenizer.str_to_int` → `{'!': 0, '"': 1, ... 'painted': 418, ...}`
* `tokenizer.int_to_str` → `{0: '!', 1: '"', ... 418: 'painted', ...}`

Estas estructuras residen en la memoria heap y son accedidas por referencia a través del objeto `tokenizer`.

**Complejidad:**

* Construcción del diccionario inverso: **O(n)** donde *n* es el tamaño del vocabulario.

---

### **2)**

```python
text = """Its the last he painted, you know, "Mrs. Gisburn said with pardonable pride."""
```

**Propósito:**
Define el texto que será convertido en tokens y luego a IDs.

**Detalles:**

* Uso de triple comillas `"""..."""` permite escribir cadenas multilínea sin necesidad de escapes.
  Aquí se usa en una sola línea, pero sigue siendo válido.
* El texto incluye:

  * palabras (`Its`, `the`, `last`, `painted`, `know`, `said`, etc.),
  * signos de puntuación (`,`, `"`, `.`),
  * un nombre propio con mayúscula (`Mrs. Gisburn`).

**Nota importante:**
El token `"Its"` (sin apóstrofe) podría **no existir en el vocabulario** si el texto de entrenamiento original solo contenía `"It's"`.
Dado que `SimpleTokenizerV1` **no maneja palabras desconocidas**, esto puede causar un `KeyError`.

---

### **3)**

```python
ids = tokenizer.encode(text)
```

**Propósito:**
Convierte el texto en una secuencia de enteros (*token IDs*) siguiendo el vocabulario cargado.

---

#### **Etapas internas del método `encode`:**

1. **Tokenización inicial**

   ```python
   preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
   ```

   * Divide el texto en palabras, puntuaciones y espacios.
   * Cada separador se conserva como elemento separado.
   * Ejemplo intermedio:

     ```
     ['Its', ' ', 'the', ' ', 'last', ' ', 'he', ' ', 'painted', ',', ' ', 'you', ' ', 'know', ',', ' ', '"', 'Mrs', '.', ' ', 'Gisburn', ' ', 'said', ' ', 'with', ' ', 'pardonable', ' ', 'pride', '.', '', '']
     ```

2. **Limpieza**

   ```python
   preprocessed = [item.strip() for item in preprocessed if item.strip()]
   ```

   * Elimina cadenas vacías y espacios redundantes.
   * Resultado limpio:

     ```
     ['Its', 'the', 'last', 'he', 'painted', ',', 'you', 'know', ',', '"', 'Mrs', '.', 'Gisburn', 'said', 'with', 'pardonable', 'pride', '.']
     ```

3. **Conversión a IDs**

   ```python
   ids = [self.str_to_int[s] for s in preprocessed]
   ```

   * Busca cada token en el diccionario `str_to_int`.
   * Si todos los tokens están presentes en `vocab`, se obtiene una lista de enteros:

     ```
     [315, 27, 46, 89, 418, 3, 57, 108, 3, 1, 302, 7, 410, 85, 92, 600, 612, 7]
     ```

     *(Los valores son ilustrativos; dependen del vocabulario real.)*
   * Si un token no existe (p. ej., `"Its"` en lugar de `"It's"`), Python lanzará:

     ```
     KeyError: 'Its'
     ```

**Razonamiento del error:**

* El vocabulario proviene del cuento *The Verdict*, que probablemente no contiene `"Its"`.
* La versión `"It's"` (con apóstrofe) sí estaría registrada como un token distinto.
* Por tanto, el tokenizador V1 falla con palabras fuera del conjunto de entrenamiento.

**Complejidad:**

* Búsqueda en diccionario para *n* tokens → **O(n)** promedio, ya que cada acceso hash es O(1).

---

### **4)**

```python
print(ids)
```

**Propósito:**
Muestra la lista numérica resultante, útil para validar la conversión.

* Si todos los tokens existen: imprime los **índices numéricos** del texto.
  Ejemplo hipotético:

  ```
  [315, 27, 46, 89, 418, 3, 57, 108, 3, 1, 302, 7, 410, 85, 92, 600, 612, 7]
  ```
* Si hay palabras desconocidas: no imprime nada y muestra un **KeyError** interrumpiendo la ejecución.

---

### **Conclusión operativa**

| Paso                       | Acción                    | Resultado                     | Complejidad |
| -------------------------- | ------------------------- | ----------------------------- | ----------- |
| `SimpleTokenizerV1(vocab)` | Crea mapeos token↔ID      | Diccionarios en memoria       | O(n)        |
| `encode(text)`             | Divide, limpia, convierte | Lista de IDs                  | O(m)        |
| `print(ids)`               | Visualiza los IDs         | Validación de la codificación | O(m)        |

---

### **Limitación clave evidenciada**

El **modelo V1 no soporta palabras desconocidas**.
Por tanto, si `KeyError: 'Its'` ocurre, se debe usar `SimpleTokenizerV2`, que reemplaza tokens no vistos con `<|unk|>` (token de desconocido), asegurando compatibilidad con cualquier texto de entrada.


In [37]:
print(tokenizer.decode(ids))

It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


Explicación exhaustiva:

---

### **1)**

```python
print(tokenizer.decode(ids))
```

**Propósito:**
Reconstruir texto legible a partir de la secuencia de identificadores numéricos `ids` obtenidos con `encode()`, y mostrarlo en pantalla.

---

## **Desglose interno del método `decode()`**

El método definido en la clase `SimpleTokenizerV1`:

```python
def decode(self, ids):
    text = ' '.join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

### **Etapa 1 — Reconversión de IDs a tokens**

```python
[self.int_to_str[i] for i in ids]
```

* Usa el diccionario inverso `int_to_str`, construido durante la inicialización:

  ```python
  self.int_to_str = {i: s for s, i in vocab.items()}
  ```
* Para cada entero `i` en la lista `ids`, obtiene el token textual original asociado.

**Ejemplo:**

```
ids = [315, 27, 46, 89, 418, 3, 57, 108, 3, 1, 302, 7, 410, 85, 92, 600, 612, 7]
↓
['Its', 'the', 'last', 'he', 'painted', ',', 'you', 'know', ',', '"', 'Mrs', '.', 'Gisburn', 'said', 'with', 'pardonable', 'pride', '.']
```

* Si algún ID no está en el diccionario (poco probable, ya que `encode()` solo usa IDs válidos), Python lanza `KeyError`.

* Complejidad temporal: **O(n)** en el número de IDs.

---

### **Etapa 2 — Unión de tokens con espacios**

```python
text = ' '.join([...])
```

* Concatena todos los tokens, separándolos con un **espacio**.
* El resultado inicial conserva un espacio entre cada token, incluso antes de los signos de puntuación.

**Ejemplo intermedio:**

```
'Its the last he painted , you know , " Mrs . Gisburn said with pardonable pride .'
```

* Esta forma contiene **espacios no naturales** antes de comas, puntos o comillas.

---

### **Etapa 3 — Corrección tipográfica de espacios**

```python
text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
```

**Análisis de la expresión regular:**

* `re.sub(patrón, reemplazo, texto)` busca todas las coincidencias de `patrón` y las sustituye por `reemplazo`.

* Patrón: `r'\s+([,.?!"()\'])'`

  * `\s+` → uno o más espacios.
  * `([,.?!"()'])` → grupo de captura que coincide con cualquier signo de puntuación listado:
    coma, punto, interrogación, exclamación, comillas, paréntesis o apóstrofe.

* Reemplazo: `r'\1'`

  * Sustituye toda la coincidencia (espacios + signo) por **solo el signo**, manteniendo lo que se capturó entre paréntesis.
  * Resultado: elimina los espacios que preceden a la puntuación.

**Ejemplo de transformación:**

```
'Its the last he painted , you know , " Mrs . Gisburn said with pardonable pride .'
↓
'Its the last he painted, you know, "Mrs. Gisburn said with pardonable pride.'
```

**Resultado:**

* Se eliminan espacios antes de los signos, pero se conservan los espacios correctos entre palabras.
* El texto recupera una forma natural y gramaticalmente correcta.

---

### **Etapa 4 — Retorno y visualización**

```python
return text
```

* Devuelve la cadena corregida al punto de llamada.
* `print(tokenizer.decode(ids))` muestra el resultado final.

**Salida esperada:**

```
Its the last he painted, you know, "Mrs. Gisburn said with pardonable pride."
```

---

## **Resumen de flujo de datos**

| Fase | Entrada                  | Operación                               | Salida                         | Complejidad |
| ---- | ------------------------ | --------------------------------------- | ------------------------------ | ----------- |
| 1    | `ids` (lista de enteros) | Mapea IDs → tokens                      | lista de strings               | O(n)        |
| 2    | lista de tokens          | Une con `' '.join()`                    | texto con espacios redundantes | O(n)        |
| 3    | texto con espacios       | `re.sub()` elimina espacios incorrectos | texto limpio                   | O(n)        |
| 4    | texto limpio             | `print()`                               | visualización                  | O(1)        |

---

## **Limitación funcional**

* Si algún token ID no tiene correspondencia en `int_to_str`, el método genera un error.
* `SimpleTokenizerV1` **no distingue** entre comillas de apertura y cierre, ni conserva saltos de línea o tabulaciones originales.
* No maneja tokens especiales como `<|endoftext|>` o `<|unk|>`.
* El resultado textual está diseñado para **coherencia lingüística básica**, no para reconstrucción exacta de formato.

---

**En síntesis:**
`decode()` implementa el paso inverso de `encode()` y ejemplifica el concepto de **reversibilidad parcial de tokenización**: a partir de una secuencia de índices numéricos, se puede recrear texto humano-legible mediante un mapeo inverso y una corrección superficial de espacios, etapa fundamental antes de evaluar la calidad de generación de un LLM.


In [39]:
# Aquí esperamos un error:

text = 'Hi, do you lik tea?' # Hi es una palabra no contenida en el vocabulario generado en lso pasos anteriores
print(tokenizer.encode(text))

KeyError: 'Hi'

Explicación exhaustiva:

---

### **1)**

```python
text = 'Hi, do you lik tea?'
```

**Propósito:**
Definir una nueva cadena de texto para comprobar el comportamiento del tokenizador cuando aparecen **palabras fuera del vocabulario**.

**Contexto técnico:**
El vocabulario `vocab` se generó a partir del cuento *The Verdict* de Edith Wharton.
Ese corpus no contiene todas las palabras del inglés; por ejemplo, es probable que no incluya:

* `"Hi"` (saludo coloquial),
* `"lik"` (error ortográfico de `"like"`).

Por tanto, estas palabras estarán **fuera del conjunto de tokens conocidos**.

**Implicación:**
El tokenizador actual (`SimpleTokenizerV1`) no maneja “out-of-vocabulary” (OOV) tokens.
Intentar convertir un token desconocido a su ID provocará una excepción de tipo `KeyError`.

---

### **2)**

```python
print(tokenizer.encode(text))
```

**Propósito:**
Llamar al método `encode()` para tokenizar y convertir el texto en IDs numéricos, e imprimir el resultado.

---

## **Flujo interno dentro de `encode()`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

---

### **Etapa 1 — División del texto**

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
```

* Usa una expresión regular para dividir el texto por signos de puntuación y espacios, manteniendo los separadores.
* Resultado intermedio aproximado:

  ```
  ['Hi', ',', '', ' ', 'do', ' ', 'you', ' ', 'lik', ' ', 'tea', '?', '']
  ```

---

### **Etapa 2 — Limpieza**

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

* Elimina espacios y cadenas vacías, dejando solo tokens válidos.
* Resultado limpio:

  ```
  ['Hi', ',', 'do', 'you', 'lik', 'tea', '?']
  ```

---

### **Etapa 3 — Conversión a IDs**

```python
ids = [self.str_to_int[s] for s in preprocessed]
```

* Intenta acceder a cada token dentro del diccionario `self.str_to_int` (el vocabulario).
* El tokenizador busca:

  * `vocab['Hi']`
  * `vocab[',']`
  * `vocab['do']`
  * `vocab['you']`
  * `vocab['lik']`
  * `vocab['tea']`
  * `vocab['?']`

**Resultado técnico:**

* Si **todos** los tokens están presentes, devuelve una lista de enteros (`List[int]`).
* En este caso, `"Hi"` y `"lik"` **no existen** en el vocabulario.
* En cuanto el intérprete intenta acceder a `self.str_to_int['Hi']`, el diccionario lanza:

  ```python
  KeyError: 'Hi'
  ```

---

### **Etapa 4 — Impresión**

* La ejecución se interrumpe antes de llegar a `print()`.
* Python muestra la traza del error:

  ```
  KeyError: 'Hi'
  ```

---

## **Análisis del error**

**Tipo de error:**
`KeyError` — ocurre cuando se intenta acceder a una clave inexistente en un diccionario.

**Razón interna:**

* En `SimpleTokenizerV1`, el método `encode()` no incluye ningún manejo de excepciones.
* El acceso directo `self.str_to_int[s]` depende de que todos los tokens estén presentes.
* Python, al no encontrar la clave `'Hi'` en la tabla hash interna del diccionario `self.str_to_int`, lanza la excepción.

**Comportamiento esperado (salida):**

```
KeyError: 'Hi'
```

---

## **Solución conceptual (adelanto del siguiente paso del libro)**

Para manejar palabras desconocidas, se usa una versión extendida del tokenizador:

```python
class SimpleTokenizerV2:
    def encode(self, text):
        ...
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
```

* Introduce un **token especial `<|unk|>`** (*unknown token*).
* Cada palabra fuera del vocabulario se reemplaza automáticamente por ese símbolo.
* Así, el método nunca falla y puede codificar cualquier texto.

**Ejemplo con `SimpleTokenizerV2`:**

```
text = 'Hi, do you lik tea?'
↓
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?']
↓
[1160, 3, 242, 57, 1160, 642, 10]
```

*(donde `1160` es el ID asignado a `<|unk|>` en el vocabulario extendido)*

---

## **Resumen**

| Etapa        | Acción                  | Resultado            | Complejidad | Observación                  |   |                    |
| ------------ | ----------------------- | -------------------- | ----------- | ---------------------------- | - | ------------------ |
| Tokenización | `re.split`              | Palabras y signos    | O(n)        | Separa espacios y puntuación |   |                    |
| Limpieza     | `strip` + filtro        | Tokens válidos       | O(n)        | Elimina vacíos               |   |                    |
| Mapeo a IDs  | Búsqueda en diccionario | `KeyError` en `'Hi'` | O(n)        | Sin manejo OOV               |   |                    |
| Solución     | Usar `<                 | unk                  | >` token    | Codificación robusta         | — | Implementado en V2 |

---

**Conclusión:**
El comando `print(tokenizer.encode(text))` con `SimpleTokenizerV1` falla porque el tokenizador no reconoce “Hi” ni “lik”.
Este comportamiento es **intencional** en la versión V1 del libro: su propósito es demostrar la necesidad de **ampliar el vocabulario** o **introducir un mecanismo de manejo de palabras desconocidas**, lo que se resolverá en la siguiente versión (`SimpleTokenizerV2`).


# **2.4 Adding special context tokens** (Agregando tokens de contexto especiales)

In [41]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


Explicación exhaustiva, línea por línea:

---

### **1)**

```python
all_tokens = sorted(list(set(preprocessed)))
```

**Propósito:**
Construir la base del **nuevo vocabulario** de tokens, eliminando duplicados y ordenándolos alfabéticamente.

**Desglose técnico:**

* `set(preprocessed)`

  * Convierte la lista `preprocessed` en un **conjunto** (`set`), eliminando repeticiones.
  * Los conjuntos en Python están implementados como **tablas hash**, lo que permite verificar y almacenar unicidad en tiempo promedio **O(1)** por operación.
  * Resultado: todos los tokens únicos del corpus.

* `list(...)`

  * Convierte el conjunto de nuevo en una lista indexable.
  * Esto es necesario porque los conjuntos son desordenados y no soportan orden ni indexación.

* `sorted(...)`

  * Ordena lexicográficamente la lista resultante según el código Unicode de cada carácter.
  * El algoritmo usado es **Timsort** (estable, O(n log n)).
  * El resultado es reproducible (misma ordenación cada vez), lo cual es crucial para que los IDs sean consistentes en diferentes ejecuciones.

**Resultado:**
Una lista `all_tokens` con todos los tokens únicos del texto base (`preprocessed`), ordenados de manera determinista.

---

### **2)**

```python
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
```

**Propósito:**
Agregar dos **tokens especiales** al vocabulario.

**Desglose conceptual:**

* `.extend(lista)`

  * Añade varios elementos al final de la lista existente (modificación in-place).
  * Complejidad O(k), donde *k* es el número de elementos añadidos (aquí, 2).

**Tokens agregados:**

1. **`<|endoftext|>`**

   * Marca el **final de una secuencia textual**.
   * Se usa durante el entrenamiento y la generación de texto para indicar cuándo detener la predicción.
   * Análogo al token `[EOS]` (*End of Sequence*) en otros modelos.

2. **`<|unk|>`**

   * Representa cualquier **token desconocido** (*unknown token*).
   * Es el símbolo que sustituirá palabras no vistas durante la fase de entrenamiento (manejo OOV, *Out-Of-Vocabulary*).
   * En la práctica, este token permite que el modelo funcione sobre cualquier texto, incluso si aparecen palabras nuevas.

**Resultado:**
`all_tokens` contiene ahora:

```
[... 'yourself', '<|endoftext|>', '<|unk|>']
```

Si el vocabulario original tenía *N* tokens, ahora tendrá *N + 2*.

---

### **3)**

```python
vocab = {token: integer for integer, token in enumerate(all_tokens)}
```

**Propósito:**
Generar un **diccionario de mapeo** entre cada token y un identificador numérico único (índice entero).

**Desglose técnico:**

* `enumerate(all_tokens)`

  * Devuelve un iterador que produce pares `(índice, token)`, donde el índice empieza en 0.
  * Ejemplo:

    ```
    0: '!', 1: '"', 2: "'", ..., 1127: '<|endoftext|>', 1128: '<|unk|>'
    ```

* La comprensión de diccionario:

  ```python
  {token: integer for integer, token in enumerate(all_tokens)}
  ```

  * Invierte el orden del par, de `(índice, token)` a `(token, índice)`.
  * Cada token textual se convierte en **clave**, y su posición en **valor entero**.
  * Implementación interna: el diccionario (`dict`) es una tabla hash con búsqueda promedio O(1).

**Importancia:**

* Este diccionario constituye el **vocabulario formal del modelo**.
* El token `<|unk|>` permitirá asignar un valor por defecto a cualquier palabra fuera del vocabulario conocido.
* El token `<|endoftext|>` servirá como marcador de final de secuencia en el entrenamiento autoregresivo del LLM.

---

### **4)**

```python
print(len(vocab.items()))
```

**Propósito:**
Verificar el tamaño total del vocabulario extendido.

**Detalles técnicos:**

* `vocab.items()` devuelve una *vista* (`dict_items`) con todos los pares `(token, id)` del diccionario.
* `len(vocab.items())` cuenta cuántos pares existen.
* Este valor equivale al **número total de tokens únicos + 2** (por los tokens especiales añadidos).

**Ejemplo de salida:**

```
1130
```

si el vocabulario original tenía 1128 tokens únicos.

**Complejidad temporal:**

* `len()` sobre una estructura `dict` o `dict_items` es O(1), ya que el tamaño se almacena internamente en el encabezado del objeto.

---

### **Resumen funcional**

| Línea                       | Acción                   | Resultado              | Complejidad |
| --------------------------- | ------------------------ | ---------------------- | ----------- |
| `set(preprocessed)`         | Eliminar duplicados      | Tokens únicos          | O(n)        |
| `sorted(list(...))`         | Ordenar tokens           | Lista ordenada         | O(n log n)  |
| `.extend([...])`            | Añadir tokens especiales | Lista +2 elementos     | O(1)        |
| Diccionario por comprensión | Crear mapeo token→ID     | Diccionario `vocab`    | O(n)        |
| `len(vocab.items())`        | Contar tokens totales    | Tamaño del vocabulario | O(1)        |

---

### **Concepto general**

Este bloque crea un **vocabulario robusto**, apto para usarse en la versión mejorada del tokenizador (`SimpleTokenizerV2`):

* Garantiza consistencia (orden determinista).
* Soporta tokens fuera del vocabulario (`<|unk|>`).
* Integra delimitadores de secuencia (`<|endoftext|>`).
* Es la base para entrenar embeddings en modelos LLM de tipo GPT o Transformer decoder.

En síntesis:

> Aquí se define el *espacio discreto completo de símbolos* que el modelo será capaz de representar, asegurando que todo texto —conocido o no— pueda codificarse de forma numérica sin errores.


In [42]:
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


Explicación exhaustiva, línea por línea:

---

### **1)**

```python
for i, item in enumerate(list(vocab.items())[-5:]):
```

**Propósito:**
Recorrer e imprimir los **últimos cinco elementos** del diccionario `vocab`, mostrando los tokens y sus identificadores numéricos más altos (generalmente los tokens añadidos al final, como `<|endoftext|>` y `<|unk|>`).

---

#### **Desglose técnico:**

##### a) `vocab.items()`

* Devuelve una **vista de elementos** del diccionario (`dict_items`), que contiene todos los pares `(clave, valor)` del diccionario `vocab`.

  * Cada elemento es una tupla: `(token, id_entero)`.
  * Ejemplo:

    ```
    ('yourself', 1127), ('<|endoftext|>', 1128), ('<|unk|>', 1129)
    ```

* En Python 3.7+, los diccionarios **preservan el orden de inserción**, por lo que los últimos elementos son los últimos tokens añadidos a `all_tokens`.

##### b) `list(vocab.items())`

* Convierte la vista `dict_items` en una **lista indexable**.
* Esto permite aplicar slicing (`[-5:]`) para acceder a los últimos cinco elementos.
* Complejidad temporal: **O(n)**, ya que se copian todas las entradas del diccionario a una nueva lista.

##### c) `[-5:]`

* Es un **slice** (rebanado) que selecciona los últimos cinco elementos de la lista.
* Ejemplo:

  ```python
  list(vocab.items())[-5:]
  # → [('your', 1125), ('yourself', 1126), ('<|endoftext|>', 1127), ('<|unk|>', 1128)]
  ```
* Si el diccionario tiene menos de 5 elementos, devuelve todos.

##### d) `enumerate(...)`

* Añade un contador `i` que empieza en 0 por defecto.

* Devuelve pares `(i, item)`, donde:

  * `i` es el índice del elemento en el bucle,
  * `item` es la tupla `(token, id)`.

* Ejemplo:

  ```
  0 ('your', 1125)
  1 ('yourself', 1126)
  2 ('<|endoftext|>', 1127)
  3 ('<|unk|>', 1128)
  ```

**Complejidad general del bucle:**

* O(n) por la conversión `list(vocab.items())`, pero solo recorre 5 elementos en la iteración.

---

### **2)**

```python
print(item)
```

**Propósito:**
Imprimir cada tupla `(token, id)` generada por el bucle.

* Cada línea de salida mostrará un par token–índice del vocabulario.
* Ejemplo de salida real:

  ```
  ('your', 1125)
  ('yourself', 1126)
  ('<|endoftext|>', 1127)
  ('<|unk|>', 1128)
  ```

**Detalles técnicos de `print`:**

* `print()` convierte cada objeto en su representación de texto (`str()` o `repr()`) y lo envía al flujo estándar de salida (`sys.stdout`).
* Cada llamada termina en salto de línea `\n` por defecto.

---

### **Resumen funcional**

| Etapa            | Acción                    | Resultado              | Complejidad |
| ---------------- | ------------------------- | ---------------------- | ----------- |
| `vocab.items()`  | Obtener pares (token, id) | vista `dict_items`     | O(1)        |
| `list(...)`      | Convertir a lista         | lista indexable        | O(n)        |
| `[-5:]`          | Tomar últimos 5           | sublista               | O(1)        |
| `enumerate(...)` | Añadir índice             | pares (i, (token, id)) | O(1)        |
| `print(item)`    | Mostrar resultado         | salida en consola      | O(1)        |

---

### **Resultado conceptual**

Este fragmento sirve como verificación visual del **final del vocabulario** y confirma que los **tokens especiales** (`<|endoftext|>`, `<|unk|>`) fueron correctamente añadidos y asignados con los índices más altos.

En contexto, los últimos valores impresos deben ser similares a:

```
('your', 1125)
('yourself', 1126)
('<|endoftext|>', 1127)
('<|unk|>', 1128)
```

Esto confirma que:

* El vocabulario está ordenado lexicográficamente.
* Los dos tokens especiales fueron añadidos al final de la lista.
* Los índices son consecutivos y únicos.

En síntesis: este bucle actúa como **validación final de la integridad del vocabulario** antes de proceder a la implementación del `SimpleTokenizerV2`.


# **Listing 2.4 A simple text tokenizer that handles unknown words**
# **Listado 2.4 Un tokenizador de texto simple que maneja palabras desconocidas**

In [43]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text


Explicación exhaustiva del diseño y comportamiento de `SimpleTokenizerV2`, línea por línea y a nivel conceptual:

---

## **Definición general**

```python
class SimpleTokenizerV2:
```

**Propósito:**
Implementar una versión **tolerante a palabras desconocidas** (*Out-Of-Vocabulary*, OOV) del tokenizador simple desarrollado previamente (`SimpleTokenizerV1`).

**Contexto técnico:**

* La versión anterior lanzaba un `KeyError` cuando un token no existía en el vocabulario.
* Esta nueva versión introduce un **mecanismo de respaldo** (`<|unk|>`) que permite procesar cualquier entrada textual.
* Representa la transición entre un tokenizador experimental y uno funcionalmente robusto para entrenamiento real de modelos.

---

## **1) Constructor**

```python
def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()}
```

### **Propósito:**

Inicializar los mapeos **token → id** y **id → token** para permitir conversión bidireccional.

### **Detalles técnicos:**

* `self.str_to_int = vocab`

  * Guarda una referencia al diccionario base que mapea cadenas (tokens) a índices enteros.
  * Este diccionario contiene ahora también los tokens especiales `<|endoftext|>` y `<|unk|>`.
  * Acceso promedio: **O(1)** (tabla hash).

* `self.int_to_str = {i: s for s, i in vocab.items()}`

  * Inversión de claves y valores para decodificación.
  * Construye un nuevo diccionario: cada id entero apunta a su token textual.
  * Ejemplo:

    ```
    {0: '!', 1: '"', ..., 1127: '<|endoftext|>', 1128: '<|unk|>'}
    ```
  * Complejidad de construcción: **O(n)** donde *n* es el número de tokens.
  * Requiere recorrer el vocabulario completo una vez.

**Resultado:**
Dos estructuras complementarias que permiten conversión directa entre representación textual y numérica.

---

## **2) Método `encode()`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

### **Propósito:**

Convertir texto crudo (`str`) en una secuencia de identificadores numéricos (`List[int]`), asignando `<|unk|>` a los tokens no presentes en el vocabulario.

---

### **Etapa 1 — Tokenización base**

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
```

* Usa la misma expresión regular que en la versión V1 para segmentar texto:

  * `(...)` grupo de captura → conserva los separadores.
  * `[,.?_!"()']` → coincide con signos de puntuación básicos.
  * `|--` → detecta doble guion literal.
  * `|\s` → divide por cualquier espacio en blanco.
* Genera una lista con palabras, signos y separadores vacíos.
* Complejidad: **O(n)** sobre el número de caracteres.

---

### **Etapa 2 — Limpieza de tokens**

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

* `item.strip()` elimina espacios al inicio y al final del token.

* El `if` descarta cadenas vacías resultantes.

* Resultado: una lista limpia con palabras y signos.

* Ejemplo:

  ```
  ['Hi', ',', 'do', 'you', 'lik', 'tea', '?']
  ```

* Complejidad: **O(m)**, donde *m* = número de tokens.

---

### **Etapa 3 — Manejo de palabras desconocidas**

```python
preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
```

* Verifica para cada token si existe en el vocabulario (`self.str_to_int`):

  * Si **existe**, lo mantiene igual.
  * Si **no existe**, lo reemplaza por el token especial `<|unk|>`.
* Esto evita excepciones `KeyError` y asegura que todos los tokens puedan convertirse en IDs.
* Implementación:

  * Búsqueda hash promedio O(1) por token.
  * Complejidad total de esta etapa: O(m).

**Ejemplo:**

```python
['Hi', ',', 'do', 'you', 'lik', 'tea', '?']
↓
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?']
```

---

### **Etapa 4 — Conversión a IDs**

```python
ids = [self.str_to_int[s] for s in preprocessed]
```

* Sustituye cada token textual por su identificador numérico.

* Como todos los tokens ya están garantizados en el vocabulario, no hay errores.

* Ejemplo hipotético:

  ```
  {'<|unk|>': 1128, ',': 3, 'do': 42, 'you': 57, 'tea': 89, '?': 9}
  ↓
  [1128, 3, 42, 57, 1128, 89, 9]
  ```

* Complejidad temporal: **O(m)**.

* Complejidad espacial: O(m) para la lista de salida.

---

### **Etapa 5 — Retorno**

```python
return ids
```

* Devuelve una lista de enteros (`List[int]`).
* Esta lista es adecuada para ser convertida en tensor de entrada en frameworks de entrenamiento (PyTorch, TensorFlow).

---

## **3) Método `decode()`**

```python
def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

### **Propósito:**

Convertir una lista de identificadores numéricos en texto legible, aplicando corrección de espacios.

---

### **Etapa 1 — Reconstrucción de tokens**

```python
[self.int_to_str[i] for i in ids]
```

* Usa el diccionario inverso `int_to_str` para traducir cada ID a su token textual.
* Complejidad: **O(m)**, acceso hash promedio O(1) por elemento.
* Resultado intermedio (lista de strings).
  Ejemplo:

  ```
  ['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?']
  ```

---

### **Etapa 2 — Unión con espacios**

```python
text = " ".join([...])
```

* Concatena los tokens con un espacio como separador.

* Ejemplo:

  ```
  '<|unk|> , do you <|unk|> tea ?'
  ```

* Complejidad: O(m) respecto a longitud del texto.

---

### **Etapa 3 — Limpieza tipográfica**

```python
text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
```

* Usa una expresión regular para eliminar espacios redundantes antes de signos de puntuación.
* Patrón:

  * `\s+` → uno o más espacios.
  * `([,.?!"()'])` → grupo de signos de puntuación comunes.
* Reemplazo: `r'\1'` → mantiene solo el signo, eliminando el espacio previo.
* Ejemplo:

  ```
  '<|unk|> , do you <|unk|> tea ?'
  ↓
  '<|unk|>, do you <|unk|> tea?'
  ```
* Complejidad: O(n) sobre longitud de la cadena.

---

### **Etapa 4 — Retorno**

```python
return text
```

* Devuelve la cadena reconstruida.
* El resultado mantiene coherencia gramatical básica, aunque puede incluir tokens `<|unk|>` que representan términos desconocidos.

**Salida final:**

```
'<|unk|>, do you <|unk|> tea?'
```

---

## **4) Análisis de robustez**

| Característica      | `V1`         | `V2`       | Efecto                    |        |           |     |                    |
| ------------------- | ------------ | ---------- | ------------------------- | ------ | --------- | --- | ------------------ |
| Manejo de OOV       | ❌            | ✅          | Evita `KeyError`          |        |           |     |                    |
| Tokens especiales   | ❌            | ✅ (`<      | unk                       | >`, `< | endoftext | >`) | Cobertura completa |
| Consistencia de IDs | ✅            | ✅          | Índices fijos             |        |           |     |                    |
| Formato de salida   | Básico       | Limpio     | Espacios corregidos       |        |           |     |                    |
| Uso en modelos      | Experimental | Entrenable | Compatible con embeddings |        |           |     |                    |

---

## **5) Complejidad global**

| Operación               | Complejidad temporal | Complejidad espacial |
| ----------------------- | -------------------- | -------------------- |
| Tokenización y limpieza | O(n)                 | O(n)                 |
| Verificación OOV        | O(n)                 | O(n)                 |
| Mapeo a IDs             | O(n)                 | O(n)                 |
| Decodificación          | O(n)                 | O(n)                 |

Donde *n* es el número de tokens procesados.

---

## **6) Importancia dentro del pipeline del LLM**

El `SimpleTokenizerV2` introduce el primer **componente de tolerancia semántica** en el preprocesamiento:

* Permite generalizar la codificación a entradas arbitrarias.
* Garantiza que el modelo pueda recibir texto sin fallar.
* Establece la base para el **vocabulario cerrado** usado en la pre-entrenamiento y fine-tuning de modelos autoregresivos tipo GPT.

---

**Conclusión:**
`SimpleTokenizerV2` convierte el prototipo V1 en un tokenizador **seguro y universal**, capaz de procesar cualquier texto de entrada sin excepciones, preservando el mapeo bidireccional entre texto e índices y respetando la estructura léxica requerida por los modelos de lenguaje basados en transformadores.


In [45]:
text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'
text = ' <|endoftext|> '.join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


Explicación exhaustiva, con análisis estructural, sintáctico y funcional:

---

### **1)**

```python
text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'
```

**Propósito:**
Definir dos fragmentos de texto (`str`) que simulan **entradas independientes** dentro de un corpus o dataset de entrenamiento.

**Detalles técnicos:**

* Ambos objetos son instancias del tipo `str` de Python, almacenadas como secuencias inmutables de *code points* Unicode.
* En memoria, Python usa una representación compacta (internamente UTF-8 o UCS-2/UCS-4 según compilación), y ambas cadenas residen en el heap con sus referencias gestionadas por el recolector de basura.
* El texto está formado por palabras, espacios y signos de puntuación (`','`, `'?'`, `'.'`), los cuales más adelante serán segmentados por el tokenizador.

**Semántica en el flujo del modelo:**

* Cada `textX` representa una unidad de entrada independiente (por ejemplo, un documento, diálogo o frase).
* En entrenamiento autoregresivo, se suelen concatenar múltiples secuencias con un **token delimitador** para permitir al modelo aprender los límites de contexto.

---

### **2)**

```python
text = ' <|endoftext|> '.join((text1, text2))
```

**Propósito:**
Concatenar las dos cadenas `text1` y `text2`, insertando entre ellas el **token especial de final de texto** `<|endoftext|>`.

---

#### **Desglose técnico:**

##### a) `' <|endoftext|> '`

* Es una cadena literal que contiene:

  * un espacio inicial `' '`
  * el token especial `<|endoftext|>`
  * un espacio final `' '`
* Este token actúa como **delimitador semántico** entre fragmentos, permitiendo distinguir el final de una secuencia y el inicio de otra dentro de un mismo batch textual.
* Este token fue añadido explícitamente al vocabulario en pasos anteriores.

##### b) `.join((text1, text2))`

* `.join(iterable)` concatena los elementos del iterable especificado, separándolos por la cadena sobre la que se invoca el método.
* El iterable aquí es una **tupla**: `(text1, text2)`.

  * Las tuplas son inmutables y de acceso indexado O(1).
* `join` recorre los elementos, los concatena, y **devuelve una nueva cadena**.

  * Implementación interna: crea un buffer de tamaño exacto mediante cálculo previo de longitud total → complejidad **O(n)** sobre la suma de longitudes.

##### c) Resultado exacto:

```
'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'
```

**Estructura conceptual resultante:**

```
[secuencia_1] + [token delimitador] + [secuencia_2]
```

Este patrón es típico en *datasets concatenados de texto plano*, donde el token `<|endoftext|>` reemplaza saltos de documento o marcadores de fin de muestra.

---

### **3)**

```python
print(text)
```

**Propósito:**
Mostrar en consola el resultado de la concatenación, verificando la correcta inserción del delimitador especial.

**Salida esperada:**

```
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
```

**Detalles técnicos:**

* `print()` llama internamente a `sys.stdout.write()` con una conversión implícita `str()` sobre su argumento.
* Añade un salto de línea final (`\n`) por defecto.
* Operación O(n) en la longitud total del texto.

---

### **Análisis conceptual**

| Elemento         | Función en la arquitectura del LLM        | Implicación                                                          |                                           |                                            |
| ---------------- | ----------------------------------------- | -------------------------------------------------------------------- | ----------------------------------------- | ------------------------------------------ |
| `text1`, `text2` | Fragmentos independientes del corpus      | Simulan secuencias separadas de entrenamiento                        |                                           |                                            |
| `<               | endoftext                                 | >`                                                                   | Token delimitador aprendido por el modelo | Enseña al modelo dónde termina una muestra |
| `.join()`        | Concatenación controlada                  | Permite construir datasets textuales continuos                       |                                           |                                            |
| Resultado final  | Texto continuo con marcador de separación | Base para el entrenamiento autoregresivo sin pérdida de segmentación |                                           |                                            |

---

### **Complejidad total**

| Operación             | Complejidad temporal             | Espacial        |
| --------------------- | -------------------------------- | --------------- |
| Creación de literales | O(1)                             | O(1)            |
| `.join()`             | O(n₁ + n₂ + k)` (longitud total) | O(n₁ + n₂ + k)` |
| `print()`             | O(n₁ + n₂ + k)`                  | O(1)            |

donde *n₁* y *n₂* son las longitudes de `text1` y `text2`, y *k* la longitud del delimitador `' <|endoftext|> '`.

---

### **Significado dentro del pipeline de un LLM**

Este paso demuestra la **integración práctica de tokens especiales** dentro del flujo textual.
En los modelos tipo GPT:

* `<|endoftext|>` marca la **frontera de contexto** en el corpus concatenado.
* Durante la inferencia, el modelo puede generar este token para indicar que debe **detener la producción de texto**.
* Durante el entrenamiento, el modelo aprende que la probabilidad de este token aumenta cuando una muestra llega a su fin.

En resumen:

> Este fragmento construye una cadena compuesta que respeta la semántica de fin de texto, asegurando continuidad sintáctica y segmentación explícita entre unidades lingüísticas —un paso esencial en la preparación de corpora para el entrenamiento autoregresivo de un LLM.


In [47]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


Explicación exhaustiva del comportamiento de este bloque, tanto en nivel de ejecución como en su papel dentro del flujo del modelo de lenguaje:

---

### **1)**

```python
tokenizer = SimpleTokenizerV2(vocab)
```

**Propósito:**
Crear una instancia del tokenizador robusto (`SimpleTokenizerV2`) empleando el vocabulario extendido que incluye los tokens especiales `<|endoftext|>` y `<|unk|>`.

---

#### **Comportamiento interno:**

* Se ejecuta el **constructor** `__init__()` definido en la clase:

  ```python
  def __init__(self, vocab):
      self.str_to_int = vocab
      self.int_to_str = {i: s for s, i in vocab.items()}
  ```

* **Estructuras creadas:**

  1. `self.str_to_int`: diccionario que mapea **token → ID entero**

     * Ejemplo:

       ```
       {',': 3, '?': 9, 'Hello': 240, '<|endoftext|>': 1127, '<|unk|>': 1128}
       ```
  2. `self.int_to_str`: diccionario inverso **ID → token**, generado mediante comprensión:

     ```python
     {0: '!', 1: '"', ..., 1127: '<|endoftext|>', 1128: '<|unk|>'}
     ```

* **Complejidad:**

  * Construcción: O(n) (recorre el vocabulario completo).
  * Accesos posteriores: O(1) promedio (hash table).

* **Resultado:**
  Un tokenizador funcional capaz de codificar y decodificar cualquier texto sin errores de clave.

---

### **2)**

```python
print(tokenizer.encode(text))
```

**Propósito:**
Aplicar el proceso de **tokenización y codificación numérica** sobre la cadena `text` que contiene el delimitador `<|endoftext|>` y mostrar el resultado.

---

#### **Flujo interno dentro de `encode()`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

---

### **Etapa 1 — Tokenización regular**

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
```

**Entrada:**

```
text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'
```

**Patrón:**

* `[,.?_!"()']` → divide por puntuación básica.
* `|--` → captura doble guion literal.
* `|\s` → divide por espacios.

**Resultado inicial (lista con separadores y vacíos):**

```
['Hello', ',', '', ' ', 'do', ' ', 'you', ' ', 'like', ' ', 'tea', '?', ' ', '<|endoftext|>', ' ', 'In', ' ', 'the', ' ', 'sunlit', ' ', 'terraces', ' ', 'of', ' ', 'the', ' ', 'palace', '.', '']
```

**Complejidad:** O(n) sobre número de caracteres.

---

### **Etapa 2 — Limpieza**

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

* `strip()` elimina espacios al inicio y fin.
* `if item.strip()` descarta vacíos.
* Resultado limpio:

  ```
  ['Hello', ',', 'do', 'you', 'like', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
  ```

**Complejidad:** O(m), donde *m* = número de tokens detectados.

---

### **Etapa 3 — Sustitución de palabras desconocidas**

```python
preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
```

**Proceso:**

* Comprueba si cada token está en `self.str_to_int`.
* Si no, lo reemplaza por `<|unk|>`.

**Caso particular:**

* `"Hello"` y `"like"` probablemente **no existen** en el vocabulario del cuento *The Verdict*.
* Por tanto, serán reemplazados por `<|unk|>`.

**Resultado tras esta etapa:**

```
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

**Importante:**
El token `<|endoftext|>` **sí existe** en el vocabulario, por lo que se conserva literalmente.

---

### **Etapa 4 — Mapeo token → ID**

```python
ids = [self.str_to_int[s] for s in preprocessed]
```

* Convierte cada token textual en su identificador numérico.
* Acceso a diccionario: O(1) promedio por token.
* Complejidad total: O(m).

**Ejemplo de salida hipotética:**

```
[1128, 3, 42, 57, 1128, 89, 9, 1127, 215, 27, 390, 420, 84, 27, 500, 7]
```

donde:

| Token    | ID (ejemplo) |    |      |
| -------- | ------------ | -- | ---- |
| `<       | unk          | >` | 1128 |
| `,`      | 3            |    |      |
| `do`     | 42           |    |      |
| `you`    | 57           |    |      |
| `tea`    | 89           |    |      |
| `?`      | 9            |    |      |
| `<       | endoftext    | >` | 1127 |
| `In`     | 215          |    |      |
| `the`    | 27           |    |      |
| `palace` | 500          |    |      |
| `.`      | 7            |    |      |

---

### **Etapa 5 — Retorno y visualización**

```python
return ids
```

* Devuelve la lista de IDs enteros.
* `print()` muestra el resultado en consola como lista Python estándar.

**Salida visual esperada:**

```
[1128, 3, 42, 57, 1128, 89, 9, 1127, 215, 27, 390, 420, 84, 27, 500, 7]
```

*(Los valores dependen del orden real del vocabulario.)*

---

## **Análisis conceptual**

| Elemento   | Significado funcional        | Papel en el LLM                               |                                                          |                                               |
| ---------- | ---------------------------- | --------------------------------------------- | -------------------------------------------------------- | --------------------------------------------- |
| `<         | unk                          | >`                                            | Token de sustitución para palabras fuera del vocabulario | Permite codificar cualquier texto sin errores |
| `<         | endoftext                    | >`                                            | Delimitador explícito entre secuencias                   | Enseña al modelo los límites contextuales     |
| `encode()` | Conversión texto → índices   | Base para construir tensores de entrenamiento |                                                          |                                               |
| `vocab`    | Espacio discreto de símbolos | Define el universo léxico del modelo          |                                                          |                                               |

---

### **Complejidad global del método**

| Etapa                | Operación        | Complejidad temporal | Espacial |
| -------------------- | ---------------- | -------------------- | -------- |
| Tokenización (regex) | `re.split()`     | O(n)                 | O(n)     |
| Limpieza             | `strip` + filtro | O(n)                 | O(n)     |
| Sustitución OOV      | Búsqueda hash    | O(n)                 | O(n)     |
| Mapeo a IDs          | Diccionario hash | O(n)                 | O(n)     |

Donde *n* es el número de caracteres o tokens procesados.

---

### **Interpretación dentro del pipeline del LLM**

Este paso completa la fase de **codificación simbólica robusta**:

1. Garantiza que cualquier texto pueda transformarse en una secuencia de enteros, eliminando errores en la fase de batch tokenization.
2. Mantiene los límites entre documentos con `<|endoftext|>`.
3. Proporciona una representación **determinista y reproducible**, indispensable para entrenar embeddings o modelos de atención.

---

**Conclusión:**
La instrucción

```python
print(tokenizer.encode(text))
```

produce una secuencia de IDs enteros que representa de forma numérica la cadena

```
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
```

con manejo seguro de tokens desconocidos mediante `<|unk|>` y preservación del token `<|endoftext|>`.
Este es el paso en que el texto humano se convierte, por primera vez, en una estructura formal apta para el aprendizaje en un modelo de lenguaje.


In [48]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Explicación exhaustiva, incluyendo funcionamiento interno completo, estructura de datos implicadas y relevancia dentro del flujo de un LLM:

---

## **1)**

```python
print(tokenizer.decode(tokenizer.encode(text)))
```

**Propósito:**
Ejecutar el ciclo completo **texto → tokens → IDs → texto**, validando que el tokenizador `SimpleTokenizerV2` sea **reversible**, consistente y tolerante a palabras desconocidas (*out-of-vocabulary*, OOV).

---

## **2) Flujo general de ejecución**

La expresión se evalúa de adentro hacia afuera:

1. `tokenizer.encode(text)`
   → convierte el texto crudo en una lista de identificadores enteros.
2. `tokenizer.decode(...)`
   → traduce esos identificadores de vuelta a texto.
3. `print(...)`
   → muestra la cadena reconstruida resultante.

---

## **3) Entrada inicial**

```python
text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'
```

El texto contiene:

* palabras comunes (`do`, `you`, `tea`, `the`, etc.),
* palabras potencialmente desconocidas (`Hello`, `like`),
* un token especial `<|endoftext|>` que **sí está en el vocabulario**,
* puntuación (`?`, `,`, `.`).

---

## **4) Etapas internas de `encode()`**

El método `encode()` ejecuta:

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
ids = [self.str_to_int[s] for s in preprocessed]
return ids
```

### **Etapa 4.1 — Tokenización**

Divide el texto en palabras, signos y espacios, conservando separadores.

**Salida intermedia:**

```
['Hello', ',', 'do', 'you', 'like', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

### **Etapa 4.2 — Sustitución de palabras fuera del vocabulario**

* `"Hello"` y `"like"` no están en el vocabulario del cuento *The Verdict*.
* Se reemplazan por `<|unk|>` (token desconocido).

**Resultado limpio:**

```
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

### **Etapa 4.3 — Conversión a IDs**

Cada token se transforma en su índice entero del vocabulario (`self.str_to_int`).

**Ejemplo conceptual:**

```
[1128, 3, 42, 57, 1128, 89, 9, 1127, 215, 27, 390, 420, 84, 27, 500, 7]
```

---

## **5) Etapas internas de `decode()`**

El método `decode()`:

```python
def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

### **Etapa 5.1 — Reconstrucción desde IDs**

Convierte cada entero a su token textual original con `self.int_to_str[i]`.

**Salida intermedia (lista de tokens):**

```
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

### **Etapa 5.2 — Unión con espacios**

```python
" ".join([...])
```

Concatena tokens con un espacio entre ellos:

```
'<|unk|> , do you <|unk|> tea ? <|endoftext|> In the sunlit terraces of the palace .'
```

### **Etapa 5.3 — Limpieza tipográfica**

```python
re.sub(r'\s+([,.?!"()\'])', r'\1', text)
```

* Busca cualquier espacio antes de signos de puntuación y los elimina.
* No altera los tokens `<|unk|>` ni `<|endoftext|>`, porque no coinciden con el patrón de signos.

**Resultado limpio:**

```
'<|unk|>, do you <|unk|> tea? <|endoftext|> In the sunlit terraces of the palace.'
```

---

## **6) Etapa final — Impresión**

```python
print(...)
```

Muestra la cadena final resultante:

```
<|unk|>, do you <|unk|> tea? <|endoftext|> In the sunlit terraces of the palace.
```

---

## **7) Análisis conceptual del resultado**

| Elemento    | Significado                                                 | Función en el flujo                    |                                                                    |                                               |
| ----------- | ----------------------------------------------------------- | -------------------------------------- | ------------------------------------------------------------------ | --------------------------------------------- |
| `<          | unk                                                         | >`                                     | Representa tokens no presentes en el vocabulario (“Hello”, “like”) | Permite codificar sin error cualquier entrada |
| `,` `?` `.` | Signos de puntuación reconstruidos sin espacios incorrectos | Verifica consistencia del reensamblado |                                                                    |                                               |
| `<          | endoftext                                                   | >`                                     | Delimitador explícito entre fragmentos textuales                   | Enseña límites de contexto al modelo          |

---

## **8) Propiedades técnicas del proceso completo**

| Propiedad                           | Descripción                                                                        | Consecuencia                               |    |                                             |
| ----------------------------------- | ---------------------------------------------------------------------------------- | ------------------------------------------ | -- | ------------------------------------------- |
| **Reversibilidad parcial**          | Las palabras conocidas se reconstruyen idénticas; las desconocidas vuelven como `< | unk                                        | >` | Pérdida controlada de información semántica |
| **Determinismo**                    | La codificación y decodificación son funciones puras, sin aleatoriedad             | Resultados reproducibles                   |    |                                             |
| **Consistencia del espacio léxico** | Solo aparecen tokens del vocabulario                                               | Compatible con entrenamiento de embeddings |    |                                             |
| **Complejidad total**               | O(n) en número de tokens, tanto en encode como en decode                           | Escalable para corpus grandes              |    |                                             |

---

## **9) Significado dentro del pipeline del LLM**

Este paso valida la **coherencia bidireccional del tokenizador**, condición necesaria para:

* entrenar embeddings consistentes (`token_id → vector → token_id`);
* garantizar que cada ID corresponde a un símbolo textual válido;
* controlar el flujo de fin de secuencia mediante `<|endoftext|>`;
* permitir inferencia segura con textos arbitrarios (manejo OOV).

En entrenamiento real, este mecanismo garantiza que:

> Cualquier texto del dataset puede ser mapeado a secuencias numéricas válidas y luego reconstruido de manera legible, asegurando estabilidad en la codificación semántica del modelo.

---

## **10) Resultado final**

**Salida esperada en consola:**

```
<|unk|>, do you <|unk|> tea? <|endoftext|> In the sunlit terraces of the palace.
```

**Interpretación:**

* El tokenizador reemplazó “Hello” y “like” por `<|unk|>`.
* Preservó puntuación, estructura y delimitador `<|endoftext|>`.
* El ciclo `encode → decode` demuestra que el tokenizador es **robusto, determinista y seguro frente a OOV**, cumpliendo con los requisitos de un preprocesador de texto para un LLM autoregresivo.


# 2.5 **Byte pair encoding**
# 2.5 **Codificación de pares de bytes**

In [49]:
pip install tiktoken

In [17]:
from importlib.metadata import version
import tiktoken
print('tiktoken version: ', version('tiktoken'))


tiktoken version:  0.12.0


In [18]:
tokenizer = tiktoken.get_encoding('gpt2')


In [19]:
text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknowPlace.'
integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
print(integers)



[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 617, 2954, 2197, 27271, 13]


In [20]:
strings = tokenizer.decode(integers)
print(strings)


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknowPlace.


In [21]:
text_akw = 'Akwirw ier'
integers_akw = tokenizer.encode(text_akw)
print(integers_akw)

[33901, 86, 343, 86, 220, 959]


# Ejercicio recomendado del libro:
Usa el tokenizador BPE de la biblioteca tiktoken sobre las palabras desconocidas «Akwirw ier» e imprime los IDs de los tokens individuales. Luego, llama a la función decode para cada uno de los enteros resultantes en esta lista para reproducir el mapeo mostrado en la Figura 2.11. Por último, llama al método decode con la lista de IDs de los tokens para verificar si puede reconstruir la entrada original, «Akwirw ier».

In [22]:
for token_id in integers_akw:
  token_text = tokenizer.decode([token_id])
  print(token_text)

Ak
w
ir
w
 
ier


In [23]:
strings_akw = tokenizer.decode(integers_akw)
print(strings_akw)

Akwirw ier
